In [5]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import calendar
import json
import csv
import numpy as np
from selenium import webdriver
import time
import math
import re
import datetime
from selenium.webdriver.chrome.options import Options
import warnings
from selenium.webdriver.support.ui import Select

In this Notebook we will generate competitions. Since new gas stations do not arrive every second we can use this dataframe throughout our analysis, assuming that it is fix, so we will not need to rerun this script again. In the end we want a dataframe witht the following columns:

| City | 1 km competition | 1 km competition | 5 km competition | 10 km competition | 20 km competition | 30 km competition | 40 km competition | 

We can include the number of competitions, which will only be the lenght of these lists.

In [15]:
warnings.filterwarnings('ignore', category = DeprecationWarning)
chrome_options = Options()
chrome_options.add_argument('--headless')

driver = webdriver.Chrome('/home/csaba/Downloads/chromedriver', options=chrome_options)
driver.get('https://holtankoljak.net/')
time.sleep(4)

#Click on 'OK tovább pop-up':
driver.find_element('xpath','/html/body/div[1]/div/div/div/div[2]/div/button[2]').click()
time.sleep(5)

mutasd_buttons = driver.find_elements('xpath', "//input[@type='submit' and @value='Mutasd']")
for button in mutasd_buttons:
    if button.is_enabled() and button.is_displayed():
        mutasd_button = button
        break

time.sleep(3)

toggles = driver.find_elements('xpath', '/html/body/div[2]/div[2]/div/div[1]/form[1]/div[1]/select')
fuel_toggle = Select(toggles[0])
fuel_toggle.select_by_index(1)

city_toggle = Select(toggles[1])
radius_toggle = Select(toggles[2])

city_toggle_options = city_toggle.options
radius_toggle_options = radius_toggle.options

competitions = pd.DataFrame(columns = ['City', '1km', 'Number of competition 1km','5km', 'Number of competition 5km','10km', 'Number of competition 10km','20km', 'Number of competition 20km','30km', 'Number of competition 30km','40km', 'Number of competition 40km'])

for index in range(1,len(city_toggle_options)):
    
    #we need to obtain in each for loop our buttons and toggle bars
    
    time.sleep(3)
    mutasd_buttons = driver.find_elements('xpath', "//input[@type='submit' and @value='Mutasd']")
    for button in mutasd_buttons:
        if button.is_enabled() and button.is_displayed():
            mutasd_button = button
            break
    time.sleep(3)
    
    toggles = driver.find_elements(By.TAG_NAME, 'select')

    city_toggle = Select(toggles[1])  #obtain the tuggles
    city_toggle.select_by_index(index)
    city = city_toggle.first_selected_option.text #this is the city
    
    time.sleep(3)
    
    competition_data_row = []
    competition_data_row.append(city)
    
    for radius in range(1, len(radius_toggle_options)):
        
        time.sleep(5)
        mutasd_buttons = driver.find_elements('xpath', "//input[@type='submit' and @value='Mutasd']")
        for button in mutasd_buttons:
            if button.is_enabled() and button.is_displayed():
                mutasd_button = button
                break
        time.sleep(3)
        
        toggles = driver.find_elements(By.TAG_NAME, 'select')
        radius_toggle = Select(toggles[2])  #obtain the tuggles
        radius_toggle.select_by_index(radius)
        radius = radius_toggle.first_selected_option.text

        mutasd_button.click()
        time.sleep(5)
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        
        rows = driver.find_elements('xpath', '/html/body/section/div[1]/div[2]/div/div[2]/div/div/table/tbody/tr[2]/td/div[1]/table/tbody/tr')
        competitions_in_radius = []
                
        for row in rows:
            
            try:
                competitions_in_radius.append(row.find_element('xpath', './/td[3]').text)
            except:
                pass
        
        competition_data_row.append(competitions_in_radius)
        competition_data_row.append(len(competitions_in_radius))

        driver.back()
        
    competitions.loc[len(competitions)] = competition_data_row

driver.quit()
competitions.to_csv('competitions.csv', index=False, sep = '\t')

   City                    1km  Number of competition 1km  \
0  Abda  [Abda, Bécsi út 186.]                          1   

                                        5km  Number of competition 5km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1.]                          2   

                                                10km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1., Győr,...   

   Number of competition 10km  \
0                          24   

                                                20km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1., Győr,...   

   Number of competition 20km  \
0                          37   

                                                30km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1., Győr,...   

   Number of competition 30km  \
0                          53   

                                                40km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1., Győr,...   

   Number of competition 40km  
0                          68  


KeyboardInterrupt: 